In [8]:
import subprocess
from rag import load_model, load_data, inference
from IPython.display import Markdown, display


In [4]:
harvey, harvey_embed = load_model()
litt, litt_embed = load_model()
judge, judge_embed = load_model()

harvey_engine = load_data(harvey, harvey_embed)
litt_engine = load_data(litt, litt_embed)
judge_engine = load_data(judge, judge_embed)

Generating embeddings: 100%|██████████| 3/3 [00:00<00:00,  4.02it/s]


In [5]:
epoch = 10

initial_case = "I think that my apartment is the best apartment in the world"

for i in range(epoch):
    prosecutor_prompt = f"Create atleast 10 of the best rebuttal arguments to each of the following arguments: {initial_case}"
    prosecutor_response = inference(litt_engine, prosecutor_prompt)
    judge_prompt = f"Look through every argument and using prior knowledge, determine if each argument is valid or invalid. Only output the valid arguments: {prosecutor_response}"
    judge_response = inference(judge_engine, judge_prompt)
    harvey_prompt = f"Create rebuttals that directly addresses each of the following arguments: {judge_response}"
    harvey_response = inference(harvey_engine, harvey_prompt)
    initial_case = harvey_response
    print(f"Epoch {i+1}: {harvey_response}")


print(initial_case)

Epoch 1: <IPython.core.display.Markdown object>
Epoch 2: <IPython.core.display.Markdown object>
Epoch 3: <IPython.core.display.Markdown object>
Epoch 4: <IPython.core.display.Markdown object>
Epoch 5: <IPython.core.display.Markdown object>
Epoch 6: <IPython.core.display.Markdown object>
Epoch 7: <IPython.core.display.Markdown object>
Epoch 8: <IPython.core.display.Markdown object>
Epoch 9: <IPython.core.display.Markdown object>
Epoch 10: <IPython.core.display.Markdown object>
<IPython.core.display.Markdown object>


In [16]:
initial_case_str = str(initial_case)
display(Markdown(initial_case))

TypeError: Markdown expects text, not <IPython.core.display.Markdown object>